# Example 1.1: What is random vibrations?
In this example, we will discuss how stochastic vibrations are different from deterministic vibrations. The stochastic behaviour of a linear dynamic system originates from the uncertainty of the dynamic loads. If we do not know the loads, we do not know the response either. 

This situation is not different from what we are used to from static analysis since we rarely know the static loads precisely. We instead use what we know about the statistical properties of the static loads to determine design loads with a selected annual probability of exceedance. This approach is possible since the largest static loads will give the largest static response and associated load effect.  

We use a similar approach when designing structures subjected to uncertain dynamic loads. Still, since the response depends on the instantaneous load and the load history, we need to use an approach where the load history and thus dynamic magnification are considered.

This example will consider a simple two degree of freedom system subjected to deterministic (known) and stochastic (uncertain) loads. The example illustrates how stochastic and deterministic dynamic loads and vibrations are different.

In [1]:
# Import necessary packages and modules
import numpy as np
from scipy.linalg import eig
from functions import *

## Define structural properties
We start by defining the structural properties of a two-degree of freedom dynamic system

In [2]:
m = 1.0 # Mass of each story
k = 100.0 # Stiffness
MM = np.eye(2)*m # Mass matrix
KK = np.array(([[2, -1], [-1, 1]])) # Stiffness matrix
# Calculate modes and frequencies
lam,v = eig(KK,MM) #Solve eigenvalue problem using scipy 
v[:,0] = v[:,0]/np.max(v[:,0]) #Normalize the eigenvactor
v[:,1] = v[:,1]/np.max(v[:,1])
f = np.real(lam)**0.5/2/np.pi #Natural frequencies in Hz
omega = f*2*np.pi # Natural frequencies in rad/s
zeta = np.array(([[5.0, 5.0]]))/100
omega.shape
# Rayleigh damping
print(omega.shape)
#omega[0,0]*omega[0,1]

#alpha1 = 2*omega[0,0]*omega[0,1]*(zeta[0,1]*omega[0,0]-zeta[0,0]*omega[0,1])/(omega[0,0]**2-omega[0,1]**2)

#alpha2 = 2*(zeta[0,0]*omega[0,0]-zeta[0,1]*omega[0,1])/(omega[0,0]**2-omega[0,1]**2)




(2,)
